<b><font color="red">* To access the model, please load the models from .pth files in the "model_usage.ipynb". <br>** With some initial weight, certain models may not converge.</font></b>

In [37]:
import pandas as pd
import torch
import numpy as np
from sklearn import metrics
import bentoml
import json
import torch.nn as nn
import torch.optim as optim
import time
import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader

## Load, Preprocess, & Split data

In [38]:
df = pd.read_csv("./Data/features_3_sec.csv")
df

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock.00099.5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,rock.00099.6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,rock.00099.7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,rock.00099.8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


In [39]:
def extract_name(x):
    return x.split('.')[0]+'.'+x.split('.')[1]

def number(x):
    # obtain the ID of the song
    return int(x.split('.')[1])

def label(x):
    # convert string label into integer category
    d = {
        "blues": 0,
        "classical":1,
        "country": 2,
        "disco":3,
        "hiphop":4,
        "jazz": 5,
        "metal":6,
        "pop":7,
        "reggae":8,
        "rock":9
    }
    return d[x]

In [40]:
song = pd.DataFrame()
song['song'] = df['filename'].apply(extract_name)
song['label'] = df['label'].apply(label)
song['number'] = song['song'].apply(number)
df['number'] = df['filename'].apply(number)
# drop unused columns
df.drop(columns=['length', 'tempo', 'perceptr_mean','perceptr_var','label','filename'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9990 entries, 0 to 9989
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   chroma_stft_mean         9990 non-null   float64
 1   chroma_stft_var          9990 non-null   float64
 2   rms_mean                 9990 non-null   float64
 3   rms_var                  9990 non-null   float64
 4   spectral_centroid_mean   9990 non-null   float64
 5   spectral_centroid_var    9990 non-null   float64
 6   spectral_bandwidth_mean  9990 non-null   float64
 7   spectral_bandwidth_var   9990 non-null   float64
 8   rolloff_mean             9990 non-null   float64
 9   rolloff_var              9990 non-null   float64
 10  zero_crossing_rate_mean  9990 non-null   float64
 11  zero_crossing_rate_var   9990 non-null   float64
 12  harmony_mean             9990 non-null   float64
 13  harmony_var              9990 non-null   float64
 14  mfcc1_mean              

No need to handle missing values
- The original dataset is clean
- The features extraction code drop the row with missing values. Therefire the newly exrtacted dataset will also be clean

In [51]:
# this df will be used for ensembling result of different segments of the same song to further improve the accuracy
song

,song,label,number
0,blues.00000,0,0
1,blues.00000,0,0
2,blues.00000,0,0
3,blues.00000,0,0
4,blues.00000,0,0
...,...,...,...
9985,rock.00099,9,99
9986,rock.00099,9,99
9987,rock.00099,9,99
9988,rock.00099,9,99


In [53]:
# Split the dataframe by song ID
# This ensure that all segments of a song remains in the same dataset for ensemling apporach
x_train = df[df['number'] >= 20]
x_val = df[df['number'] < 20]
song_train = song[song['number'] >= 20]
song_val = song[song['number'] < 20]
x_val

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,number
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,0
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,0
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,0
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,0
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9187,0.510478,0.067812,0.137442,0.000471,2382.837801,95890.852456,2157.320527,47301.659445,4813.250639,5.454456e+05,...,18.839457,-6.600465,44.321831,5.637548,17.616732,-5.278373,20.246199,-0.877959,15.256322,19
9188,0.517307,0.063758,0.104017,0.002828,2252.497236,94959.815941,2071.915479,41463.920038,4548.640700,4.018423e+05,...,26.437086,-8.344719,15.496710,2.172762,64.985870,-1.632867,14.844873,2.336645,23.016424,19
9189,0.422443,0.082976,0.126436,0.001261,2239.760106,262924.983760,2138.593210,82567.079338,4649.018555,1.158886e+06,...,17.965687,-7.647231,29.996981,2.986220,26.933788,-3.504234,16.753084,0.204434,23.066519,19
9190,0.436105,0.079302,0.129293,0.000675,2066.207203,186924.659732,2105.389165,74889.709411,4415.548941,9.786898e+05,...,19.505196,-6.679996,30.482504,2.490437,23.382385,-4.498179,22.776510,-1.911705,16.353846,19


In [43]:
x_train

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,number
200,0.274168,0.095494,0.082787,0.001400,1430.583940,90388.046462,1829.558353,67355.973409,2980.526593,5.130056e+05,...,75.780464,-8.392806,71.304054,-2.363670,83.485107,-1.457948,146.627640,0.821828,117.664154,20
201,0.285800,0.093292,0.064784,0.001630,1333.040315,84946.113534,1908.361033,97767.675787,2832.692871,1.157030e+06,...,37.606544,-6.054728,42.716873,-4.472951,67.612930,2.411800,41.252251,8.174487,29.740971,20
202,0.291340,0.095959,0.063447,0.000599,1274.004446,51841.907876,1867.110641,77624.137099,2651.565505,9.155874e+05,...,28.155912,-5.323036,36.842255,-1.868089,45.987854,-1.691187,57.396564,2.435588,48.908344,20
203,0.335422,0.096567,0.063272,0.001353,1492.238144,101402.408007,1955.914076,79519.599563,3231.139949,7.922385e+05,...,67.726364,-4.983536,40.922844,-2.800988,30.736599,-5.176728,28.020010,0.217755,20.596766,20
204,0.307678,0.097503,0.054217,0.000619,1401.144165,58983.959478,2052.317605,67266.507714,3319.923002,8.012564e+05,...,49.347980,-3.173147,34.055576,0.417186,41.342144,-0.406688,72.350197,6.398019,72.829262,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,3015.559458,8.479527e+05,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,99
9986,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,3746.694524,1.170890e+06,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,99
9987,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,2442.362154,2.602871e+06,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,99
9988,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,4313.266226,4.968878e+05,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,99


In [44]:
song_train

,song,label,number
200,blues.00020,0,20
201,blues.00020,0,20
202,blues.00020,0,20
203,blues.00020,0,20
204,blues.00020,0,20
...,...,...,...
9985,rock.00099,9,99
9986,rock.00099,9,99
9987,rock.00099,9,99
9988,rock.00099,9,99


In [45]:
song_val

,song,label,number
0,blues.00000,0,0
1,blues.00000,0,0
2,blues.00000,0,0
3,blues.00000,0,0
4,blues.00000,0,0
...,...,...,...
9187,rock.00019,9,19
9188,rock.00019,9,19
9189,rock.00019,9,19
9190,rock.00019,9,19


In [46]:
x_train= x_train.drop(columns=['number'])
x_val= x_val.drop(columns=['number'])
song_train= song_train.drop(columns=['number'])
song_val= song_val.drop(columns=['number'])

In [47]:
# Use one_hot enconding for the single column of label
# NN with output layer of 10 nodes
y_train = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_train["label"])), num_classes=10))
y_train
print(y_train.shape)

torch.Size([7994, 10])


C:\Users\User\AppData\Local\Temp\ipykernel_16692\564108120.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_train["label"])), num_classes=10))


In [48]:
y_val = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_val["label"])), num_classes=10))
y_val

C:\Users\User\AppData\Local\Temp\ipykernel_16692\1782615230.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(torch.nn.functional.one_hot(torch.tensor(list(song_val["label"])), num_classes=10))


tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]])

In [49]:
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val.values, dtype=torch.float32)

In [50]:
# Move tensors to cuda and convert it to float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_train_tensor=x_train_tensor.to(device)
x_val_tensor=x_val_tensor.to(device)
y_train=y_train.to(device)
y_val=y_val.to(device)
y_train =y_train.float()
y_val =y_val.float()

# Model training

## Useful function

In [15]:
def cal_song_acc(song, pred):
    # Aggregated the results of each segment by summing
    # Take max as the predicted class
    copy_song = song.reset_index()
    grouped = copy_song.groupby('song')
    true_labels = list(grouped["label"].mean().astype(int))
    # print(true_labels)
    aggregated_list = []
    max_locs_list = []
    for group_name, group_indices in grouped.groups.items():
        group_values = copy_song.loc[group_indices, 'label']
        group_list_indices = list(group_values.index)
        group_list = [pred[i] for i in group_list_indices]
        # Calculate sum or average for each column of the 2D list
        group_sum = np.sum(group_list, axis=0)  # or np.mean(group_list, axis=0) for average
        aggregated_list.append(group_sum)
        # Find the location of the maximum value in the aggregated array
        max_loc = np.argmax(group_sum)
        max_locs_list.append(max_loc)
    # print(max_locs_list)
    # print(metrics.accuracy_score(true_labels, max_locs_list))
    return metrics.accuracy_score(true_labels, max_locs_list)

def cal_acc(song, pred):
    max_locs_list = []
    for row in pred:
        max_index = torch.argmax(row).item()
        max_locs_list.append(max_index)
    return metrics.accuracy_score(list(song["label"]), max_locs_list)

def save_model(model, model_info, model_name, train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration):
    bentoml.pytorch.save_model(model_name, model)
    print("saved")
    logs = dict()
    logs["params"] = model_info
    logs["train_loss"] = train_loss
    logs["val_loss"] = val_loss
    logs["train_acc"] = train_acc
    logs["train_song_acc"] = train_song_acc
    logs["val_acc"] = val_acc
    logs["val_song_acc"] = val_song_acc
    logs["duration"] = duration
    latest_file_path = f"../../../bentoml/models/{model_name}/latest"
    with open(latest_file_path, "r") as file:
        tag = file.read().strip()
    file_path = f"json_log\{model_name}_{tag}"
    # Save dictionary as JSON file
    with open(file_path, 'w') as json_file:
        json.dump(logs, json_file)
    print("logged")

def train(num_epochs, model, optimizer, criterion, batch_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TensorDataset(x_train_tensor, y_train)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x=x.to(device)
            y=y.to(device)
            y=y.float()
            # print(x.device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
        model.eval()
        with torch.no_grad():
            train_loss = 0.0
            val_loss = 0.0
            
            outputs = model(x_train_tensor)
            train_loss += criterion(outputs, y_train).item()
            train_song_acc = cal_song_acc(song_train, outputs.to("cpu"))
            train_acc = cal_acc(song_train, outputs.to("cpu"))
            
            outputs = model(x_val_tensor)
            val_loss += criterion(outputs, y_val).item()
            val_song_acc = cal_song_acc(song_val, outputs.to("cpu"))
            val_acc = cal_acc(song_val, outputs.to("cpu"))
        print(f"Epoch {epoch + 1}:")
        print(f"  Train Loss: {train_loss}")
        print(f"  Train Song Accuracy: {train_song_acc}")
        print(f"  Train Accuracy: {train_acc}")
        print(f"  Validation Loss: {val_loss}")
        print(f"  Validation Song Accuracy: {val_song_acc}")
        print(f"  Validation Accuracy: {val_acc}")
    end_time = time.time()
    duration = end_time - start_time
    return train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration

## ANN

### Run 0
- First try with simpler model
- Increase layers and neurons until finding the first model that converge

In [26]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(54, 108)
        self.fc2 = nn.Linear(108, 108)
        self.fc3 = nn.Linear(108, 54)
        self.fc4 = nn.Linear(54, 10)  # Adjust output size based on the number of classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [32]:
model_info = {
    "name": "ANN",
    "arch": "54-108-108-108-108-108-108-54-10",
    "criterion": "BCELoss",
    "lr":0.001,
    "epoch": 100,
    "batch_size":128
}

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 9.728347778320312
  Train Song Accuracy: 0.08
  Train Accuracy: 0.09131848886664999
  Validation Loss: 9.749310493469238
  Validation Song Accuracy: 0.095
  Validation Accuracy: 0.10370741482965933
Epoch 2:
  Train Loss: 9.606378555297852
  Train Song Accuracy: 0.11
  Train Accuracy: 0.10032524393294971
  Validation Loss: 9.608125686645508
  Validation Song Accuracy: 0.105
  Validation Accuracy: 0.11923847695390781
Epoch 3:
  Train Loss: 8.460689544677734
  Train Song Accuracy: 0.10625
  Train Accuracy: 0.12246685013760321
  Validation Loss: 8.315160751342773
  Validation Song Accuracy: 0.105
  Validation Accuracy: 0.12324649298597194
Epoch 4:
  Train Loss: 9.330231666564941
  Train Song Accuracy: 0.14875
  Train Accuracy: 0.15686765073805353
  Validation Loss: 9.35190486907959
  Validation Song Accuracy: 0.16
  Validation Accuracy: 0.15180360721442887
Epoch 5:
  Train Loss: 8.612332344055176
  Train Song Accuracy: 0.10625
  Train Accuracy: 0.11883912934701026
  

In [33]:
save_model(model, model_info, "ANN", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN' to lowercase: 'ann'.


saved
logged


### Run 1
- Try increasing epoch to see if the accuracy continue to increase or not

In [192]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(54, 108)
        self.fc2 = nn.Linear(108, 108)
        self.fc3 = nn.Linear(108, 54)
        self.fc4 = nn.Linear(54, 10)  # Adjust output size based on the number of classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [177]:
model_info = {
    "name": "ANN",
    "arch": "54-108-108-108-108-108-108-54-10",
    "criterion": "BCELoss",
    "lr":0.001,
    "epoch": 300,
    "batch_size":128
}

In [173]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 17.124181747436523
  Train Song Accuracy: 0.1
  Train Accuracy: 0.09982486865148861
  Validation Loss: 17.05103302001953
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 2:
  Train Loss: 9.623085975646973
  Train Song Accuracy: 0.1325
  Train Accuracy: 0.11358518889166876
  Validation Loss: 9.68923282623291
  Validation Song Accuracy: 0.135
  Validation Accuracy: 0.12374749498997996
Epoch 3:
  Train Loss: 9.615131378173828
  Train Song Accuracy: 0.085
  Train Accuracy: 0.09532149111833875
  Validation Loss: 9.711003303527832
  Validation Song Accuracy: 0.09
  Validation Accuracy: 0.10070140280561123
Epoch 4:
  Train Loss: 9.60765266418457
  Train Song Accuracy: 0.1175
  Train Accuracy: 0.10632974731048286
  Validation Loss: 9.722546577453613
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10270541082164329
Epoch 5:
  Train Loss: 9.268818855285645
  Train Song Accuracy: 0.15875
  Train Accuracy: 0.14498373780335252
  Valida

In [178]:
save_model(model, model_info, "ANN", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN' to lowercase: 'ann'.


saved
logged


## Run 2
- Try models with deeper architecture

In [283]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(54, 108)
        self.fc2 = nn.Linear(108, 108)
        self.fc3 = nn.Linear(108, 216)
        self.fc4 = nn.Linear(216, 216)
        self.fc5 = nn.Linear(216, 108)
        self.fc6 = nn.Linear(108, 54)
        self.fc7 = nn.Linear(54, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.sigmoid(self.fc7(x))
        return x

In [206]:
model_info = {
    "name": "ANN",
    "arch": "54-108-108-108-216-216-108-54-10",
    "criterion": "BCELoss",
    "lr":0.0009,
    "epoch": 300,
    "batch_size":128
}

In [202]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.0009)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 9.368306159973145
  Train Song Accuracy: 0.14
  Train Accuracy: 0.12747060295221416
  Validation Loss: 9.392301559448242
  Validation Song Accuracy: 0.12
  Validation Accuracy: 0.1407815631262525
Epoch 2:
  Train Loss: 9.672358512878418
  Train Song Accuracy: 0.12
  Train Accuracy: 0.10695521641230923
  Validation Loss: 9.768942832946777
  Validation Song Accuracy: 0.115
  Validation Accuracy: 0.10170340681362726
Epoch 3:
  Train Loss: 6.501351356506348
  Train Song Accuracy: 0.125
  Train Accuracy: 0.12784588441331
  Validation Loss: 6.362363338470459
  Validation Song Accuracy: 0.125
  Validation Accuracy: 0.11372745490981964
Epoch 4:
  Train Loss: 8.91861343383789
  Train Song Accuracy: 0.11
  Train Accuracy: 0.12334250688016012
  Validation Loss: 8.970918655395508
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.11823647294589178
Epoch 5:
  Train Loss: 0.3785158097743988
  Train Song Accuracy: 0.20125
  Train Accuracy: 0.16875156367275457
  Validation

In [203]:
save_model(model, model_info, "ANN", train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration)

Converting 'ANN' to lowercase: 'ann'.


saved
logged


## Run 3
- Add another layer to see if the accuracy can be further increased this way.
- The initial weights result in saddle points.
- Try Xavier/He weight initialization.

In [291]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(54, 108)
        self.fc2 = nn.Linear(108, 216)
        self.fc3 = nn.Linear(216, 432)
        self.fc4 = nn.Linear(432, 216)
        self.fc5 = nn.Linear(216, 108)
        self.fc6 = nn.Linear(108, 54)
        self.fc7 = nn.Linear(54, 10)
        for layer in [self.fc1, self.fc2, self.fc3, self.fc4, self.fc5, self.fc6, self.fc7]:
            init.xavier_uniform_(layer.weight)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.sigmoid(self.fc7(x))
        return x

In [251]:
model_info = {
    "name": "ANN",
    "arch": "54-108-108-108-216-216-108-54-10",
    "criterion": "BCELoss",
    "lr":0.0009,
    "epoch": 300,
    "batch_size":128,
    "optimizer":"Adam",
    "activation":"Sigmoid"
}

In [293]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 41.993995666503906
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 42.0240478515625
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.09869739478957916
Epoch 2:
  Train Loss: 41.993995666503906
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 42.0240478515625
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.09869739478957916
Epoch 3:
  Train Loss: 41.993995666503906
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 42.0240478515625
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.09869739478957916
Epoch 4:
  Train Loss: 41.993995666503906
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 42.0240478515625
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.09869739478957916
Epoch 5:
  Train Loss: 41.993995666503906
  Train Song Accuracy: 0.1
  Train Accuracy: 0.10007505629221916
  Validation Loss: 42

KeyboardInterrupt: 

## Run 2 - rerun for testing
- Rerun run number 2 to verify the hypothesis that random weight initialization can result in saddle points.
- Maintain the same model architecture and hyperparameters.

In [294]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(54, 108)
        self.fc2 = nn.Linear(108, 108)
        self.fc3 = nn.Linear(108, 216)
        self.fc4 = nn.Linear(216, 216)
        self.fc5 = nn.Linear(216, 108)
        self.fc6 = nn.Linear(108, 54)
        self.fc7 = nn.Linear(54, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.sigmoid(self.fc7(x))
        return x

In [206]:
model_info = {
    "name": "ANN",
    "arch": "54-108-108-108-216-216-108-54-10",
    "criterion": "BCELoss",
    "lr":0.0009,
    "epoch": 300,
    "batch_size":128
}

In [295]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
# print(device)
# Define loss function and optimizer
criterion = nn.BCELoss()  # Cross-entropy loss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.0009)
num_epochs = 300
batch_size = 128
train_loss, val_loss, train_acc, train_song_acc, val_acc, val_song_acc, duration = train(num_epochs, model, optimizer, criterion, batch_size)

Epoch 1:
  Train Loss: 17.985980987548828
  Train Song Accuracy: 0.1
  Train Accuracy: 0.09982486865148861
  Validation Loss: 17.956680297851562
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 2:
  Train Loss: 17.99905014038086
  Train Song Accuracy: 0.1
  Train Accuracy: 0.09982486865148861
  Validation Loss: 17.9830322265625
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 3:
  Train Loss: 25.97005271911621
  Train Song Accuracy: 0.09875
  Train Accuracy: 0.10020015011258444
  Validation Loss: 25.944656372070312
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 4:
  Train Loss: 34.0004997253418
  Train Song Accuracy: 0.1
  Train Accuracy: 0.09982486865148861
  Validation Loss: 33.98797607421875
  Validation Song Accuracy: 0.1
  Validation Accuracy: 0.10020040080160321
Epoch 5:
  Train Loss: 34.0004997253418
  Train Song Accuracy: 0.1
  Train Accuracy: 0.09982486865148861
  Validation Loss:

KeyboardInterrupt: 

In [58]:
load_model = bentoml.pytorch.load_model('ann:dbpp2xag2kp2jpap', device_id="cuda")
torch.save(load_model.state_dict(), r"model_state\all_2.pth")